In [179]:
Name = "Md. Sakib Bin Alam"
ID = "st122574"

## Assignment 2

Constraint: Only use our code (not other code....)

1. I guess you already try a bigger corpus
2. I guess you already try window size 2
3. I guess you already have skipgram, skipgram(neg), cbow, glove

Do this:
1. Compare them based on syntactic accuracy and semantic accuracy, similar to how is done in https://nlp.stanford.edu/pubs/glove.pdf (see Table 2) - NO NEED to try 1000 or 300 embed size.....I just want you to learn how to do experiment.....
2. Try to find a correlation with just ONE similarity dataset (which humans judge how similar is two words.....)

Point criteria:
0:  Not done
1: ok
2: with comments / explanation / figures just like how Chaky explain thing.....

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

## 1. Load Data

In [2]:
# import os
# os.environ['http_proxy']  = 'http://192.41.170.23:3128'
# os.environ['https_proxy'] = 'http://192.41.170.23:3128'
# #import dataset

In [3]:
with open('questions-words.txt') as file:
    data = [line.strip() for line in file.readlines() if line[0] != ':']

In [4]:
print("length:", len(data))
print(data[:10])

length: 19544
['Athens Greece Baghdad Iraq', 'Athens Greece Bangkok Thailand', 'Athens Greece Beijing China', 'Athens Greece Berlin Germany', 'Athens Greece Bern Switzerland', 'Athens Greece Cairo Egypt', 'Athens Greece Canberra Australia', 'Athens Greece Hanoi Vietnam', 'Athens Greece Havana Cuba', 'Athens Greece Helsinki Finland']


## 2. Prepare train data

In [5]:
# for s in data:
#     print(s)

In [6]:
# Tokenization
data_tokenized = [sent.split(" ") for sent in data]
data_tokenized[:5]

[['Athens', 'Greece', 'Baghdad', 'Iraq'],
 ['Athens', 'Greece', 'Bangkok', 'Thailand'],
 ['Athens', 'Greece', 'Beijing', 'China'],
 ['Athens', 'Greece', 'Berlin', 'Germany'],
 ['Athens', 'Greece', 'Bern', 'Switzerland']]

In [7]:
# Numericalize

flatten = lambda l: [item for sublist in l for item in sublist]
vocabs  = list(set(flatten(data_tokenized))) 

In [8]:
len(vocabs)

905

In [9]:
vocabs[:10]

['Banjul',
 'Minsk',
 'Europe',
 'taller',
 'going',
 'Lima',
 'generating',
 'Chilean',
 'Alaska',
 'run']

In [10]:
word2index = {v: idx for idx, v in enumerate(vocabs)}

In [11]:
vocabs.append('<UNK>')
word2index['<UNK>'] = len(vocabs) - 1

In [12]:
index2word = {v:k for k, v in word2index.items()}

# Skipgram

In [13]:
skipgrams = []

#for each corpus
for sent in data_tokenized:
    #for each sent
    for i in range(1, len(sent) - 1): #start from 1 to second last
        center_word = word2index[sent[i]]
        outside_words = [word2index[sent[i-1]], word2index[sent[i+1]]]  #window_size = 1
        for o in outside_words:
            skipgrams.append([center_word, o])

skipgrams[:10]

[[847, 753],
 [847, 819],
 [819, 847],
 [819, 746],
 [847, 753],
 [847, 193],
 [193, 847],
 [193, 506],
 [847, 753],
 [847, 391]]

In [14]:
def random_batch(batch_size, skip_grams):
    random_inputs = []
    random_labels = []
    random_index = np.random.choice(range(len(skip_grams)), batch_size, replace=False) #randomly pick without replacement
        
    for i in random_index:
        random_inputs.append([skip_grams[i][0]])  # target, e.g., 2
        random_labels.append([skip_grams[i][1]])  # context word, e.g., 3
            
    return np.array(random_inputs), np.array(random_labels)

### Test Function

In [15]:
#testing the method
batch_size = 2 # mini-batch size
input, label = random_batch(batch_size, skipgrams)

print("Input: ", input)
print("Target: ", label)

Input:  [[151]
 [292]]
Target:  [[125]
 [268]]


## Model Skipgram

In [16]:
#the model will accept three vectors - u_o, v_c, u_w
#u_o - vector for outside words
#v_c - vector for center word
#u_w - vectors of all vocabs

class Skipgram(nn.Module):
    
    def __init__(self, voc_size, emb_size):
        super(Skipgram, self).__init__()
        self.embedding_center_word  = nn.Embedding(voc_size, emb_size)
        self.embedding_outside_word = nn.Embedding(voc_size, emb_size)
    
    def forward(self, center_word, outside_word, all_vocabs):
        #center_word, outside_word: (batch_size, 1)
        #all_vocabs: (batch_size, voc_size)
        
        #convert them into embedding
        center_word_embed  = self.embedding_center_word(center_word)     #(batch_size, 1, emb_size)
        outside_word_embed = self.embedding_outside_word(outside_word)   #(batch_size, 1, emb_size)
        all_vocabs_embed   = self.embedding_outside_word(all_vocabs)     #(batch_size, voc_size, emb_size)
        
        #bmm is basically @ or .dot , but across batches (i.e., ignore the batch dimension)
        top_term = outside_word_embed.bmm(center_word_embed.transpose(1, 2)).squeeze(2)
        #(batch_size, 1, emb_size) @ (batch_size, emb_size, 1) = (batch_size, 1, 1) ===> (batch_size, 1)
        
        top_term_exp = torch.exp(top_term)  #exp(uo vc)
        #(batch_size, 1)
        
        lower_term = all_vocabs_embed.bmm(center_word_embed.transpose(1, 2)).squeeze(2)
         #(batch_size, voc_size, emb_size) @ (batch_size, emb_size, 1) = (batch_size, voc_size, 1) = (batch_size, voc_size)
         
        lower_term_sum = torch.sum(torch.exp(lower_term), 1) #sum exp(uw vc)
        #(batch_size, 1)
        
        loss_fn = -torch.mean(torch.log(top_term_exp / lower_term_sum))
        #(batch_size, 1) / (batch_size, 1) ==mean==> scalar
        
        return loss_fn

In [17]:
#preparing all_vocabs

batch_size = 5

voc_size = len(vocabs)
voc_size

def prepare_sequence(seq, word2index):
    #map(function, list of something)
    #map will look at each of element in this list, and apply this function
    idxs = list(map(lambda w: word2index[w] if word2index.get(w) is not None else word2index["<UNK>"], seq))
    return torch.LongTensor(idxs)

all_vocabs = prepare_sequence(list(vocabs), word2index).expand(batch_size, voc_size)
all_vocabs.shape

torch.Size([5, 906])

## Train Model

In [18]:
import time

In [19]:
emb_size  = 5
model_Skipgram    = Skipgram(voc_size, emb_size)
optimizer = optim.Adam(model_Skipgram.parameters(), lr=0.0001)

In [20]:
num_epochs = 10000

train_start_time = time.time()

#for epoch
for epoch in range(num_epochs):
    start_time = time.time()
    #get random batch
    input_batch, label_batch = random_batch(batch_size, skipgrams)
    input_batch = torch.LongTensor(input_batch)
    label_batch = torch.LongTensor(label_batch)
    
    # print(input_batch.shape, label_batch.shape, all_vocabs.shape)
    
    #loss = model
    loss = model_Skipgram(input_batch, label_batch, all_vocabs)
    
    #backpropagate
    loss.backward()
    
    #update alpha
    optimizer.step()
    
    total_time = time.time() - start_time
    #print epoch loss
    if (epoch + 1) % 1000 == 0:
        print(f"Epoch {epoch+1} | Loss: {loss:.6f} | Time: {total_time}")
        
total_training_time = time.time() - train_start_time
print("Total trainig time:", total_training_time)

Epoch 1000 | Loss: 8.073089 | Time: 0.016032695770263672
Epoch 2000 | Loss: 7.385178 | Time: 0.020861387252807617
Epoch 3000 | Loss: 9.267972 | Time: 0.016358613967895508
Epoch 4000 | Loss: 6.753740 | Time: 0.016054868698120117
Epoch 5000 | Loss: 6.438174 | Time: 0.01591181755065918
Epoch 6000 | Loss: 6.462528 | Time: 0.015711545944213867
Epoch 7000 | Loss: 5.139174 | Time: 0.015846967697143555
Epoch 8000 | Loss: 6.303550 | Time: 0.015986919403076172
Epoch 9000 | Loss: 8.994142 | Time: 0.015226602554321289
Epoch 10000 | Loss: 6.228723 | Time: 0.016278505325317383
Total trainig time: 161.51635265350342


# Skipgram Negative Sampling

## Unigram distribution

In [21]:
from collections import Counter

z = 0.001
word_count = Counter(flatten(data_tokenized))
num_total_words = sum([c for w, c in word_count.items()])
num_total_words

78176

In [22]:
unigram_table = []

for v in vocabs:
    uw = word_count[v]/num_total_words
    uw_alpha = uw ** 0.75
    uw_alpha_dividebyz = int(uw_alpha / z)
    unigram_table.extend([v] * uw_alpha_dividebyz)

In [23]:
def prepare_sequence(seq, word2index):
    #map(function, list of something)
    #map will look at each of element in this list, and apply this function
    idxs = list(map(lambda w: word2index[w] if word2index.get(w) is not None else word2index["<UNK>"], seq))
    return torch.LongTensor(idxs)

In [24]:
import random
#you don't want to pick samples = targets, basically negative samples
#k = number of negative samples - how many? they found 10 is the best
#will be run during training
#after random_batch, 
def negative_sampling(targets, unigram_table, k):
    #targets is already in id.....
    #but the unigram_table is in word....
    #1. get the batch size of this targets
    batch_size = targets.shape[0]
    neg_samples = []
    #2. for each batch
    for i in range(batch_size):
        #randomly pick k negative words from unigram_table
        target_index = targets[i].item()  #looping each of the batch....
        nsample = []
        while len(nsample) < k:
            neg = random.choice(unigram_table)
            #if this word == target, skip this word
            if word2index[neg] == target_index:
                continue
            nsample.append(neg)
        #append this word to some list
        neg_samples.append(prepare_sequence(nsample, word2index).reshape(1, -1))  #tensor[], tensor[]
    return torch.cat(neg_samples)  #tensor[[], []]

## Model Skipgram Negative Sampling

In [25]:
class SkipgramNeg(nn.Module):
    
    def __init__(self, voc_size, emb_size):
        super(SkipgramNeg, self).__init__()
        self.embedding_center_word  = nn.Embedding(voc_size, emb_size)
        self.embedding_outside_word = nn.Embedding(voc_size, emb_size)
        self.logsigmoid = nn.LogSigmoid()
        
    def forward(self, center_words, outside_words, negative_words):
        #center_words, outside_words: (batch_size, 1)
        #negative_words:  (batch_size, k)
        
        center_embed  = self.embedding_center_word(center_words)    #(batch_size, 1, emb_size)
        outside_embed = self.embedding_outside_word(outside_words)  #(batch_size, 1, emb_size)
        neg_embed     = self.embedding_outside_word(negative_words) #(batch_size, k, emb_size)
        
        uovc          =  outside_embed.bmm(center_embed.transpose(1, 2)).squeeze(2)  #(batch_size, 1)
        ukvc          = -neg_embed.bmm(center_embed.transpose(1, 2)).squeeze(2)  #(batch_size, k)
        ukvc_sum      =  torch.sum(ukvc, 1).view(-1, 1) #(batch_size, 1)
        
        loss = self.logsigmoid(uovc) + self.logsigmoid(ukvc_sum)  #(batch_size, 1) + (batch_size, 1)
                
        return -torch.mean(loss)  #scalar, loss should be scalar, to call backward()

### Test Model

In [26]:
input, label = random_batch(batch_size,skipgrams)
input_tensor = torch.LongTensor(input)  
label_tensor = torch.LongTensor(label)

In [27]:
emb_size = 2
voc_size = len(vocabs)
model = SkipgramNeg(voc_size, emb_size)

In [28]:
neg_tensor = negative_sampling(label_tensor, unigram_table, 5)

In [29]:
input_tensor.shape, label_tensor.shape

(torch.Size([5, 1]), torch.Size([5, 1]))

In [30]:
loss = model(input_tensor, label_tensor, neg_tensor)
loss

tensor(2.3530, grad_fn=<NegBackward0>)

## Train Model

In [31]:
emb_size = 5 #so we can later plot
model_SkipgramNeg  = SkipgramNeg(voc_size, emb_size)
num_neg        = 10 # num of negative sampling

optimizer = optim.Adam(model.parameters(), lr=0.001)

In [32]:
model_SkipgramNeg

SkipgramNeg(
  (embedding_center_word): Embedding(906, 5)
  (embedding_outside_word): Embedding(906, 5)
  (logsigmoid): LogSigmoid()
)

In [33]:
import time

init_time = time.time()

# Training
num_epochs = 5000
for epoch in range(num_epochs):
    
    start_time = time.time()
    
    input_batch, target_batch = random_batch(batch_size, skipgrams)
    
    #input_batch: [batch_size, 1]
    input_batch = torch.LongTensor(input_batch)
    
    #target_batch: [batch_size, 1]
    target_batch = torch.LongTensor(target_batch)
    
    #negs_batch:   [batch_size, num_neg]
    negs_batch = negative_sampling(target_batch, unigram_table, num_neg)
    
    optimizer.zero_grad()
        
    loss = model_SkipgramNeg(input_batch, target_batch, negs_batch)
    
    total_time = time.time() - start_time
    
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch {epoch+1} | Loss: {loss:.6f} | Time: {total_time}")
        
total_training_time = time.time() - train_start_time
print("Total trainig time:", total_training_time)

Epoch 1000 | Loss: 5.413135 | Time: 0.014811992645263672
Epoch 2000 | Loss: 3.879498 | Time: 0.014837980270385742
Epoch 3000 | Loss: 7.471133 | Time: 0.014683961868286133
Epoch 4000 | Loss: 4.606218 | Time: 0.014765024185180664
Epoch 5000 | Loss: 4.508828 | Time: 0.02836155891418457
Total trainig time: 300.11318278312683


# CBOW

In [34]:
cbows = []

#for each corpus
for sent in data_tokenized:
    #for each sent
    for i in range(1, len(sent) - 1): #start from 1 to second last
        center_word = word2index[sent[i]]
        
        outside_words = []
        low  = i - 1
        high = i + 1
        for j in range(low, high + 1):
            if j == i:
                continue
            outside_words.append(word2index[sent[j]])
        cbows.append([center_word, outside_words])
        
cbows[:10]

[[847, [753, 819]],
 [819, [847, 746]],
 [847, [753, 193]],
 [193, [847, 506]],
 [847, [753, 391]],
 [391, [847, 47]],
 [847, [753, 255]],
 [255, [847, 303]],
 [847, [753, 403]],
 [403, [847, 716]]]

In [35]:
def random_batch_cbow(batch_size, cbows):
    random_inputs = []
    random_labels = []
    random_index = np.random.choice(range(len(cbows)), batch_size, replace=False) #randomly pick without replacement
        
    for i in random_index:
        random_inputs.append([cbows[i][0]])  # target, e.g., 2
        random_labels.append([cbows[i][1]])  # context word, e.g., 3
            
    return np.array(random_inputs), np.array(random_labels).squeeze()

## Model CBOW

Reference:
- http://cs224d.stanford.edu/lecture_notes/notes1.pdf
- Took help from Abhinav's Assignment 1

In [36]:
class CBOW(nn.Module):
    
    def __init__(self, voc_size, emb_size):
        super(CBOW, self).__init__()
        self.emb_size = emb_size
        self.input_word  = nn.Embedding(voc_size, emb_size) #v
        self.output_word = nn.Embedding(voc_size, emb_size) #u
    
    def forward(self, center_word, outside_word, all_vocabs):
        #center_word: (batch_size, 1)
        #context_words: (batch_size, window_size * 2)
        #all_vocabs: (batch_size, voc_size)
        batch_size = center_word.shape[0]
        
        #convert them into embedding
        center_word_embed  = self.output_word(center_word)   #(batch_size, 1, emb_size)
        outside_word_embed = self.input_word(outside_word)   #(batch_size, window_size * 2, emb_size)
        all_vocabs_embed   = self.output_word(all_vocabs)    #(batch_size, voc_size, emb_size)        
        
        # mean of input word embeddings
        v_mean = torch.sum(outside_word_embed, 1) / len(outside_word) #(batch_size, emb_size)
        
        ucv = center_word_embed.bmm(v_mean.reshape(batch_size, 1, self.emb_size).transpose(1, 2)).squeeze()
        #(batch_size, 1, emb_size) @ (batch_size, emb_size, 1) = (batch_size, 1, 1) ==> (batch_size, 1)
        
        ujv = all_vocabs_embed.bmm(v_mean.reshape(batch_size, 1, self.emb_size).transpose(1, 2)).squeeze(2)
        #(batch_size, voc_size, emb_size) @ (batch_size, emb_size, 1) = (batch_size, voc_size)
        
        ujv_log_exp = torch.log(torch.exp(ujv))
        # (batch_size, voc_size) -> (batch_size,)
        
        loss_fn = - ucv + torch.sum(ujv_log_exp, 1)
        # - (batch_size, 1) + (batch_size, 1) = (batch_size, 1)
        
        return torch.mean(loss_fn)

## Training

In [37]:
emb_size = 2
model_cbow = CBOW(voc_size, emb_size)
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [38]:
model_cbow

CBOW(
  (input_word): Embedding(906, 2)
  (output_word): Embedding(906, 2)
)

In [39]:
init_time = time.time()

#for epoch
for epoch in range(num_epochs):
    start_time = time.time()
    #get random batch
    input_batch, label_batch = random_batch_cbow(batch_size, cbows)
    input_batch = torch.LongTensor(input_batch)
    label_batch = torch.LongTensor(label_batch)
    
    #loss = model
    loss = model_cbow(input_batch, label_batch, all_vocabs)
    
    #backpropagate
    loss.backward()
    
    #update alpha
    optimizer.step()
    
    total_time = time.time() - start_time
    #print epoch loss
    if (epoch + 1) % 1000 == 0:
        print(f"Epoch {epoch+1} | Loss: {loss:.6f} | Time: {total_time}")
        
total_training_time = time.time() - init_time
print("Total trainig time:", total_training_time)

Epoch 1000 | Loss: 3.804660 | Time: 0.012360334396362305
Epoch 2000 | Loss: -5.989528 | Time: 0.009416818618774414
Epoch 3000 | Loss: -2.011968 | Time: 0.00953364372253418
Epoch 4000 | Loss: -2.777016 | Time: 0.009201288223266602
Epoch 5000 | Loss: -0.894053 | Time: 0.009268045425415039
Total trainig time: 48.29684400558472


# GLOVE

## Co-occurence matrix

In [40]:
from collections import Counter

#count the frequency of each word....
X_i = Counter(flatten(data_tokenized)) #merge all list

In [41]:
gloves = []

#loop through each corpus
for sent in data_tokenized: 
    #loop through each word from 1 to n-1 (because 0 and n has no context window)
    for i in range(1, len(sent)-1):
        target  = sent[i]
        context = [sent[i+1], sent[i-1]]
        #append(i, i+1) and append(i, i-1)
        for c in context:
            gloves.append((target, c))

gloves[:10]

[('Greece', 'Baghdad'),
 ('Greece', 'Athens'),
 ('Baghdad', 'Iraq'),
 ('Baghdad', 'Greece'),
 ('Greece', 'Bangkok'),
 ('Greece', 'Athens'),
 ('Bangkok', 'Thailand'),
 ('Bangkok', 'Greece'),
 ('Greece', 'Beijing'),
 ('Greece', 'Athens')]

In [42]:
#since we have these occurrences, we can count, to make our co-occurrence matrix!!!
X_ik_skipgram = Counter(gloves)

In [43]:
#X_ik_skipgram

## Weighting function f

In [44]:
def weighting(w_i, w_j, X_ik):   #why we need w_i and w_j, because we can try its co-occurrences, if it's too big, we scale it down
    
    #check whether the co-occurrences between these two word exists???
    try:
        x_ij = X_ik[(w_i, w_j)]
    except:
        x_ij = 1  #why one, so that the probability thingy won't break...(label smoothing)
        
    #maximum co-occurrences; we follow the paper
    x_max = 100
    alpha = 0.75
    
    #if the co-occurrences does not exceed x_max, scale it down based on some alpha
    if x_ij < x_max:
        result = (x_ij/x_max) ** alpha
    else:
        result = 1 #this is the maximum probability you can have
        
    return result

In [45]:
w_i  = 'Baghdad'
w_j  = 'Iraq'
w_j2 = 'demo'

print(weighting(w_i, w_j, X_ik_skipgram))   #scales from 1 to 0.0316
print(weighting(w_i, w_j2, X_ik_skipgram))  #the paper says that f(0) = 0

0.6902356338456498
0.0


In [46]:
#now apply this weighting to all possible pairs
from itertools import combinations_with_replacement

X_ik = {} #for keeping the co-occurrences
weighting_dic = {} #for keeping all the probability after passing through the weighting function

for bigram in combinations_with_replacement(vocabs, 2):  #we need to also think its reverse
    #if this bigram exists in X_ik_skipgrams
    #we gonna add this to our co-occurence matrix
    if X_ik_skipgram.get(bigram) is not None:
        cooc = X_ik_skipgram[bigram]  #get the co-occurrence
        X_ik[bigram] = cooc + 1 #this is again basically label smoothing....(stability issues (especially when divide something))
        X_ik[(bigram[1], bigram[0])] = cooc + 1  #trick to get all pairs
    else: #otherwise, do nothing
        pass
    
    #apply the weighting function using this co-occurrence matrix thingy    
    weighting_dic[bigram] = weighting(bigram[0], bigram[1], X_ik)
    weighting_dic[(bigram[1], bigram[0])] = weighting(bigram[1], bigram[0], X_ik)

In [47]:
len(X_ik_skipgram)

37266

## Prepare train data

In [48]:
import math

def random_batch_glove(batch_size, word_sequence, gloves, X_ik, weighting_dic):
    
    #loop through this skipgram, and change it id  because when sending model, it must number
    gloves_id = [(word2index[g[0]], word2index[g[1]]) for g in gloves]
    
    #randomly pick "batch_size" indexes
    number_of_choices = len(gloves_id)
    random_index = np.random.choice(number_of_choices, batch_size, replace=False) #no repeating indexes among these random indexes
    
    random_inputs = [] #xi, wi (in batches)
    random_labels = [] #xj, wj (in batches)
    random_coocs  = [] #Xij (in batches)
    random_weighting = [] #f(Xij) (in batches)
    #for each of the sample in these indexes
    for i in random_index:
        random_inputs.append([gloves_id[i][0]]) #same reason why i put bracket here....
        random_labels.append([gloves_id[i][1]])
        
        #get cooc
        #first check whether it exists...
        pair = gloves[i]  #e.g., ('banana', 'fruit)
        try:
            cooc = X_ik[pair]
        except:
            cooc = 1 #label smoothing
            
        random_coocs.append([math.log(cooc)])  #1. why log, #2, why bracket -> size ==> (, 1)  #my neural network expects (, 1)
        
        #get weighting
        weighting = weighting_dic[pair]  #why not use try....maybe it does not exist....
        random_weighting.append(weighting)

        
    return np.array(random_inputs), np.array(random_labels), np.array(random_coocs), np.array(random_weighting)

In [49]:
batch_size = 2
input, target, cooc, weightin = random_batch_glove(batch_size, data_tokenized, gloves, X_ik, weighting_dic)

In [50]:
input, target, cooc, weightin

(array([[ 13],
        [223]]),
 array([[854],
        [239]]),
 array([[0.69314718],
        [0.69314718]]),
 array([0.05318296, 0.05318296]))

## Model GLOVE

In [51]:
class GloVe(nn.Module):
    
    def __init__(self, vocab_size,embed_size):
        super(GloVe,self).__init__()
        self.embedding_v = nn.Embedding(vocab_size, embed_size) # center embedding
        self.embedding_u = nn.Embedding(vocab_size, embed_size) # out embedding
        
        self.v_bias = nn.Embedding(vocab_size, 1)
        self.u_bias = nn.Embedding(vocab_size, 1)
        
    def forward(self, center_words, target_words, coocs, weighting):
        center_embeds = self.embedding_v(center_words) # [batch_size, 1, emb_size]
        target_embeds = self.embedding_u(target_words) # [batch_size, 1, emb_size]
        
        center_bias = self.v_bias(center_words).squeeze(1)
        target_bias = self.u_bias(target_words).squeeze(1)
        
        inner_product = target_embeds.bmm(center_embeds.transpose(1, 2)).squeeze(2)
        #[batch_size, 1, emb_size] @ [batch_size, emb_size, 1] = [batch_size, 1, 1] = [batch_size, 1]
        
        #note that coocs already got log
        loss = weighting*torch.pow(inner_product +center_bias + target_bias - coocs, 2)
        
        return torch.sum(loss)

## Training

In [52]:
voc_size   = len(vocabs)
batch_size = 2 #why?  no reason; 
emb_size   = 5 #why?  no reason; usually 50, 100, 300, but 2 so we can plot (50 can also plot, but need PCA)
model_glove      = GloVe(voc_size, emb_size)

optimizer  = optim.Adam(model.parameters(), lr=0.001)

In [53]:
model_glove

GloVe(
  (embedding_v): Embedding(906, 5)
  (embedding_u): Embedding(906, 5)
  (v_bias): Embedding(906, 1)
  (u_bias): Embedding(906, 1)
)

In [54]:
init_time = time.time()

num_epochs = 5000
#for epoch
for epoch in range(num_epochs):
    start_time = time.time()

    #get random batch
    input, target, cooc, weightin = random_batch_glove(batch_size, data_tokenized, gloves, X_ik, weighting_dic)
    input_batch    = torch.LongTensor(input)
    target_batch   = torch.LongTensor(target)
    cooc_batch     = torch.FloatTensor(cooc)
    weightin_batch = torch.FloatTensor(weightin)
    
    
    # print(input_batch.shape, label_batch.shape, cooc_batch.shape, weightin_batch)
    
    #loss = model
    loss = model_glove(input_batch, target_batch, cooc_batch, weightin_batch)
    
    #backpropagate
    loss.backward()
    
    #update alpha
    optimizer.step()
    total_time = time.time() - start_time
    if (epoch + 1) % 1000 == 0:
        print(f"Epoch {epoch+1} | Loss: {loss:.6f} | Time: {total_time}")

        
total_training_time = time.time() - init_time
print("Total trainig time:", total_training_time)

Epoch 1000 | Loss: 3.794889 | Time: 0.03260374069213867
Epoch 2000 | Loss: 12.852983 | Time: 0.03130650520324707
Epoch 3000 | Loss: 36.234222 | Time: 0.03966832160949707
Epoch 4000 | Loss: 0.972803 | Time: 0.03301095962524414
Epoch 5000 | Loss: 2.203818 | Time: 0.03419756889343262
Total trainig time: 161.96291208267212


# Part 1: Semantic Analysis

In [141]:
# model_Skipgram
# model_SkipgramNeg
# model_cbow
# model_glove

## Test data

In [108]:
with open('capital-world.txt') as file:
    test_data = [line.strip() for line in file.readlines() if line[0] != ':']

In [109]:
len(test_data)

4524

In [110]:
test_data[:5]

['abuja nigeria accra ghana',
 'abuja nigeria algiers algeria',
 'abuja nigeria amman jordan',
 'abuja nigeria ankara turkey',
 'abuja nigeria antananarivo madagascar']

In [111]:
# Tokenization
test_data_tokenized = [sent.split(" ") for sent in test_data]
test_data_tokenized[:5]

[['abuja', 'nigeria', 'accra', 'ghana'],
 ['abuja', 'nigeria', 'algiers', 'algeria'],
 ['abuja', 'nigeria', 'amman', 'jordan'],
 ['abuja', 'nigeria', 'ankara', 'turkey'],
 ['abuja', 'nigeria', 'antananarivo', 'madagascar']]

In [112]:
flatten = lambda l: [item for sublist in l for item in sublist]
test_vocabs  = list(set(flatten(test_data_tokenized))) 

In [113]:
print(len(test_vocabs))
print(test_vocabs[:5])

232
['venezuela', 'sudan', 'lilongwe', 'qatar', 'suva']


In [114]:
word2index = {v: idx for idx, v in enumerate(test_vocabs)}

In [115]:
test_vocabs.append('<UNK>')
word2index['<UNK>'] = len(test_vocabs) - 1

In [116]:
from numpy import dot
from numpy.linalg import norm

def cos_sim(a, b):
    cos_sim = dot(a, b)/(norm(a)*norm(b))
    return cos_sim

## Semantic accuracy (Skipgram)

In [117]:
#let's write a function to get embedding given a word
def get_embed(model, word):
    id_tensor = torch.LongTensor([word2index[word]])
    v_embed = model.embedding_center_word(id_tensor)
    u_embed = model.embedding_outside_word(id_tensor) 
    # print(v_embed)
    # print(u_embed)
    word_embed = (v_embed + u_embed) / 2 
    #x, y = word_embed[0][0].item(), word_embed[0][1].item()

    return word_embed

In [89]:
embeddings = []

for word in test_vocabs: #loop each unique vocab
    embeddings.append(get_embed(model_Skipgram, word))

In [309]:
len(embeddings)

233

In [310]:
test_embed = get_embed(model_Skipgram, test_vocabs[0])
test_embed

tensor([[0.6268, 0.2669, 0.0081, 0.2252, 0.6153]], grad_fn=<DivBackward0>)

In [312]:
# embeddings[:5]
# Convert the list of embeddings to a tensor
embeddings = torch.stack(embeddings)

In [323]:
# for i in range(len(embeddings) - 1):
#     for j in range(i+1, len(embeddings) - 1):
#         count = 0
#         similarity = torch.nn.CosineSimilarity(embeddings[i].detach().numpy().squeeze(), embeddings[j].detach().numpy().squeeze())
#         print(similarity)
#         if i == 0:
#             break
#         count += 1
#     #similarity = similarity/count

# similarity = similarity / len(embeddings)
    

In [303]:
# #numpy version
# from numpy import dot
# from numpy.linalg import norm

# def cos_sim(a, b):
#     # a = a.detach().numpy()
#     # b = b.detach().numpy()
#     cos_sim = dot(a, b)/(norm(a)*norm(b))
#     return cos_sim

In [304]:
# for i in range(len(embeddings) - 1):
#     for j in range(i+1, len(embeddings) - 1):
#         similarity = cos_sim(embeddings[i], embeddings[j])
#         #print(similarity)
#         if i == 0:
#             break

In [313]:
# Compute the semantic similarity using cosine similarity
similarity = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
#print(similarity)

# Compute the average similarity between all pairs of words
accuracy = similarity(embeddings, embeddings).mean()

print('Semantic accuracy:', accuracy.item())

Semantic accuracy: 1.0


## Semantic accuracy (Skipgram Negative)

In [168]:
embeddings = []

for word in test_vocabs: #loop each unique vocab
    embeddings.append(get_embed(model_SkipgramNeg, word))

In [315]:
# Convert the list of embeddings to a tensor
embeddings = torch.stack(embeddings)

In [317]:
# Compute the semantic similarity using cosine similarity
similarity = torch.nn.CosineSimilarity(dim=1, eps=1e-6)

# Compute the average similarity between all pairs of words
accuracy = similarity(embeddings, embeddings).mean()

print('Semantic accuracy:', accuracy.item())

Semantic accuracy: 1.0


## Semantic accuracy (CBOW)

In [71]:
#let's write a function to get embedding given a word
def get_embed_cbow(model, word):
    id_tensor = torch.LongTensor([word2index[word]])
    v_embed = model.input_word(id_tensor)
    u_embed = model.output_word(id_tensor) 
    word_embed = (v_embed + u_embed) / 2 
    #x, y = word_embed[0][0].item(), word_embed[0][1].item()

    return word_embed

In [167]:
embeddings = []

for word in test_vocabs: #loop each unique vocab
    embeddings.append(get_embed_cbow(model_cbow, word))

In [196]:
# Convert the list of embeddings to a tensor
embeddings = torch.stack(embeddings)

In [197]:
# Compute the semantic similarity using cosine similarity
similarity = torch.nn.CosineSimilarity(dim=1, eps=1e-6)

# Compute the average similarity between all pairs of words
accuracy = similarity(embeddings, embeddings).mean()

print('Semantic accuracy:', accuracy.item())

Semantic accuracy: 1.0


## Semantic accuracy (GLOVE)

In [118]:
#let's write a function to get embedding given a word
def get_embed_glove(model, word):
    id_tensor = torch.LongTensor([word2index[word]])
    v_embed = model.embedding_v(id_tensor)
    u_embed = model.embedding_u(id_tensor) 
    word_embed = (v_embed + u_embed) / 2 
    #x, y = word_embed[0][0].item(), word_embed[0][1].item()

    return word_embed

In [176]:
embeddings = []

for word in test_vocabs: #loop each unique vocab
    #embeddings.append(get_embed_glove(model_glove, word).squeeze())
    embed = get_embed_glove(model_glove, word).squeeze()
    embeddings.append(embed.detach().numpy())

In [177]:
# Convert the list of embeddings to a tensor
#embeddings = torch.stack(embeddings)

In [178]:
# Compute the semantic similarity using cosine similarity
#similarity = torch.nn.CosineSimilarity(dim=1, eps=1e-6)

# Compute the average similarity between all pairs of words
#accuracy = similarity(embeddings, embeddings).mean()

accuracy = cos_sim(embeddings, embeddings).mean()
print('Semantic accuracy:', accuracy)

ValueError: shapes (233,5) and (233,5) not aligned: 5 (dim 1) != 233 (dim 0)

# Part 2: Similarities

In [101]:
import torch
from scipy.stats import spearmanr

In [56]:
with open('wordsim_similarity_goldstandard.txt') as f:
    data_similarity = [line.strip().lower().split("\t") for line in f.readlines()]

In [90]:
data_similarity[:5]

[['tiger', 'cat', '7.35'],
 ['tiger', 'tiger', '10.00'],
 ['plane', 'car', '5.77'],
 ['train', 'car', '6.31'],
 ['television', 'radio', '6.77']]

In [93]:
flatten = lambda l: [item for sublist in l for item in sublist]
test_similarity_vocabs  = list((flatten(data_similarity)))

In [94]:
len(test_similarity_vocabs)

609

In [96]:
test_similarity_vocabs[:6]

['tiger', 'cat', '7.35', 'tiger', 'tiger', '10.00']

In [97]:
word2index = {v: idx for idx, v in enumerate(test_similarity_vocabs)}

In [98]:
from numpy import dot
from numpy.linalg import norm

def cos_sim(a, b):
    cos_sim = dot(a, b)/(norm(a)*norm(b))
    return cos_sim

## CBOW Similarities

spearmanr: https://www.statology.org/spearman-correlation-python/

In [103]:
# Read the similarity dataset
similarities = []
with open('wordsim_similarity_goldstandard.txt', 'r') as f:
    for line in f:
        word1, word2, similarity = line.strip().split("\t")
        word1_embedding = get_embed_cbow(model_cbow, word1).squeeze()
        word2_embedding = get_embed_cbow(model_cbow, word2).squeeze()
        cosine_similarity = cos_sim(word1_embedding.detach().numpy(), word2_embedding.detach().numpy())
        similarities.append((cosine_similarity, float(similarity)))

# Calculate the spearman correlation
spearman_correlation = spearmanr([x[0] for x in similarities], [x[1] for x in similarities])
print('Spearman correlation:', spearman_correlation)

Spearman correlation: SpearmanrResult(correlation=-0.1754386335267139, pvalue=0.4468597854046624)


## GLOVE similarities

In [105]:
# Read the similarity dataset
similarities = []
with open('wordsim_similarity_goldstandard.txt', 'r') as f:
    for line in f:
        word1, word2, similarity = line.strip().split("\t")
        word1_embedding = get_embed_glove(model_glove, word1).squeeze()
        word2_embedding = get_embed_glove(model_glove, word2).squeeze()
        cosine_similarity = cos_sim(word1_embedding.detach().numpy(), word2_embedding.detach().numpy())
        similarities.append((cosine_similarity, float(similarity)))

# Calculate the spearman correlation
spearman_correlation = spearmanr([x[0] for x in similarities], [x[1] for x in similarities])
print('Spearman correlation:', spearman_correlation)

Spearman correlation: SpearmanrResult(correlation=0.07927227144540408, pvalue=0.7326782054021117)


## Skipgram Similarities

In [106]:
# Read the similarity dataset
similarities = []
with open('wordsim_similarity_goldstandard.txt', 'r') as f:
    for line in f:
        word1, word2, similarity = line.strip().split("\t")
        word1_embedding = get_embed(model_Skipgram, word1).squeeze()
        word2_embedding = get_embed(model_Skipgram, word2).squeeze()
        cosine_similarity = cos_sim(word1_embedding.detach().numpy(), word2_embedding.detach().numpy())
        similarities.append((cosine_similarity, float(similarity)))

# Calculate the spearman correlation
spearman_correlation = spearmanr([x[0] for x in similarities], [x[1] for x in similarities])
print('Spearman correlation:', spearman_correlation)

Spearman correlation: SpearmanrResult(correlation=0.31448999491455387, pvalue=0.1649985541392634)


### Observation

In terms of correlation Skipgram and CBOW are showing some good correlation with the values 0.31 and -0.18 respectively. The reason might be that I trained Skipgram for 10000 epochs, but later on due to some issues in the puffer (it was taking too long to train) I trained other models for 5000 epochs. Although P value for all methods are showing significance. This might happen due to the less amount of data that we trained. 